In [32]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, AsyncOpenAI, OpenAIChatCompletionsModel, GuardrailFunctionOutput, input_guardrail, output_guardrail
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
from pydantic import BaseModel


In [2]:
load_dotenv(override=True)

True

### Step 1: Agent Workflow

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write professional, serious cold email."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensursing SOC2 compliance and preparing for audits, powered by AI. \
    You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write concise, to the point cold  emails."

In [4]:
google_api_key = os.getenv('GOOGLE_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"   
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)


In [9]:
sales_agent1 = Agent(
    name = 'Professional Sales Agent',
    instructions=instructions1,
    model=gemini_model
)

sales_agent2 = Agent(
    name='Engaging Sales Agent',
    instructions=instructions2,
    model=gemini_model
)

sales_agent3 = Agent(
    name = 'Busy Sales Agent',
    instructions=instructions3,
    model=gemini_model
)

In [9]:
result = Runner.run_streamed(sales_agent1, input='Write a cold sales email')
async for event in result.stream_events():
    if event.type == 'raw_response_event' and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end='', flush=True)

Subject: Streamline SOC 2 Compliance & Audit Prep with ComplAI

Dear [Name],

Maintaining SOC 2 compliance can be a significant drain on resources, often requiring complex manual processes and posing a constant challenge for growing companies. At ComplAI, we understand these hurdles and offer a solution designed to simplify and accelerate your path to achieving and maintaining SOC 2 compliance.

ComplAI is an AI-powered SaaS platform built specifically for streamlining SOC 2 compliance and audit preparation. Our platform provides:

*   **Automated Evidence Collection:** Eliminate manual data gathering with our automated evidence collection capabilities, ensuring continuous compliance.
*   **Gap Analysis & Remediation Guidance:** Identify areas of non-compliance and receive actionable recommendations to close security gaps quickly and efficiently.
*   **Simplified Audit Preparation:** Centralize documentation, automate report generation, and facilitate seamless communication with audito

In [11]:
message  = 'write a cold sales email'
with trace('Parallel cold emails'):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [result.final_output for result in results]
for output in outputs:
    print(output + '\n\n')

Subject: Streamline SOC 2 Compliance & Audit Prep with ComplAI

Dear [Recipient Name],

Maintaining SOC 2 compliance can be a significant drain on resources, often requiring manual processes and complex coordination. At ComplAI, we understand the challenges involved in achieving and maintaining this crucial certification.

ComplAI offers an AI-powered SaaS solution designed to simplify and automate your SOC 2 compliance journey. Our platform helps you:

*   **Automate Evidence Collection:** Continuously gather evidence from various sources, reducing manual effort and ensuring comprehensive coverage.
*   **Identify and Remediate Gaps:** Leverage AI-powered analysis to proactively identify potential compliance gaps and receive actionable recommendations for remediation.
*   **Streamline Audit Preparation:** Prepare for audits with confidence using our organized documentation and automated reporting features, minimizing disruption to your team.
*   **Ensure Continuous Compliance:** Monito

In [27]:
sales_picker = Agent(
    name='sales_picker',
    instructions='You pick the best cold sales email from the given options. \
         Imagine you are a customer and pick the one you are most likely to respond to. \
         Do not give and explanation; reply with the selected eamil only. ',
    model=gemini_model        
)

In [28]:
message = 'Write a cold sales email.'
with trace('Select from sales people'):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )
    outputs = [result.final_output for result in results]
    emails = 'Cold sales:\n\n'.join(outputs)
    best = await Runner.run(sales_picker, emails)
    print(f"Best sales email: \n{best.final_output}")

Best sales email: 
Subject: Streamline SOC 2 Compliance & Audit Prep with AI - ComplAI

Dear [Prospect Name],

Maintaining SOC 2 compliance can be a significant drain on resources, diverting your team's focus from core business objectives. I'm reaching out from ComplAI, a company dedicated to simplifying and accelerating the entire SOC 2 journey through the power of AI.

ComplAI is a SaaS platform that helps organizations like yours:

*   **Automate Evidence Collection:** Our AI-powered system automatically gathers and organizes the evidence needed for SOC 2 audits, significantly reducing manual effort and errors.
*   **Identify & Remediate Gaps:** Proactively identify potential compliance gaps with AI-driven risk assessments and receive actionable recommendations for remediation.
*   **Simplify Audit Preparation:** Generate audit-ready reports and documentation with just a few clicks, saving valuable time and resources.
*   **Continuous Monitoring:** Maintain ongoing compliance with r

### Step 2 and 3: Tools and Agent Interactions

In [10]:
@function_tool
def send_email(body:str):
    """ Send out an email with the given body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_KEY'))
    from_email = Email('dileep9968@gmail.com')
    to_email = To('dileep.opd20016@iiitd.ac.in')
    content = Content('Text/Plain', body)
    mail  = Mail(from_email, to_email, 'Sales email', content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {'status': 'success'}

#### This has automatically been converted into a tool with the boilerplate  json created

In [11]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1051a5080>, strict_json_schema=True, is_enabled=True)

#### You can also convert an agent into a tool

In [12]:
tool1 = sales_agent1.as_tool(tool_name='sales_agent1', tool_description='write a cold sales email')
tool1

FunctionTool(name='sales_agent1', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x140183a60>, strict_json_schema=True, is_enabled=True)

In [13]:
# We can gather all the tools together
# A tool for each of our 3 email-writing agents
# And a tool for our function to send emails

In [14]:
description = 'Write a cold sales email'
tool1 = sales_agent1.as_tool(tool_name='sales_agent1', tool_description=description)
tool2 = sales_agent2.as_tool(tool_name='sales_agent2', tool_description=description)
tool3 = sales_agent3.as_tool(tool_name='sales_agent3', tool_description=description)

tools = [tool1, tool2, tool3, send_email]
tools 


[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1401834c0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1401837e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

In [15]:
# Now it's time for our sales manager - our planning agent
instructions = 'You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
    You never generate sales emails yourself; you always use the tools. \
    You try all 3 sales_agent tools once before choosing the best one. \
    You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user.'

sales_manager = Agent(name = 'Sales Manager', instructions=instructions, tools = tools, model = gemini_model)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace('Sales manager'):
    result = await Runner.run(sales_manager, message)

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


#### Handoffs represent a way an agent can delegate to an agent, passing control to it
* 1. Handoffs and Agents-as-tool are similar
* 2. In both cases, an Agent can collaborate with another Agent
* 3. With tools, control passes back
* 4. With handoffs, control passes across

In [16]:
subject_instructions = "You can write a subject for a cold sales email. \
    You are given a message and you need to write a subject for an email that is likely to get a response. "
html_instructions = 'You can convert a text email body to an HTML email body. \
    You are gievn a text email body with might have some markdown \
    and you need to convert it to an HTML email body with simple, clear, compelling layout and design.'

subject_writer = Agent(name = 'Email subject writer', instructions=subject_instructions, model = gemini_model)
subject_tool = subject_writer.as_tool(tool_name='subject_writer', tool_description='write a subject for a cold sales email')

html_converter = Agent(name = 'HTML email body conveter', instructions=html_instructions, model=gemini_model)
html_tool = html_converter.as_tool(tool_name='html_conveter', tool_description='Convert a text email body to an HTML email body')

In [21]:
@function_tool
def send_html_email(subject:str, html_body: str)->Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key = os.getenv('SENDGRID_KEY'))
    from_email = Email('dileep9968@gmail.com')
    to_email = To('dileep.sahu@paytm.com')
    content = Content('text/html', html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {'status':'success'}

In [22]:
tools = [subject_tool, html_tool, send_html_email]
tools

[FunctionTool(name='subject_writer', description='write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x140180ae0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_conveter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_conveter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x140181260>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospe

In [35]:
instructions = "You are an email formatter and sender. You receive the body of an email to be sent. \
    You first use the subject_writer tool to write a subject for the email, then use the html_conveter tool to convert the body to HTML. \
    Finally, you use the send_html_email tool to send the eamil with the subject and HTML body."

emailer_agent = Agent(
    name = 'Email Manager',
    instructions=instructions,
    tools = tools,
    model = gemini_model,
    handoff_description='Convert and email to HTML and send it'
)

In [36]:
# Now we have 3 tools and 1 handoff
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(handoffs)

[Agent(name='Email Manager', instructions='You are an email formatter and sender. You receive the body of an email to be sent.     You first use the subject_writer tool to write a subject for the email, then use the html_conveter tool to convert the body to HTML.     Finally, you use the send_html_email tool to send the eamil with the subject and HTML body.', handoff_description='Convert and email to HTML and send it', handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x107b24b60>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'r

In [37]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales email. \
    You never generate sales emails youself; you always use the tools. \
    You can use the tools multiple times if you're not satisfied with the results from the first try. \
    You select the single best email using you own judgement of which email will be most effective. \
    After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(
    name = 'Sales Manager',
    instructions=sales_manager_instructions,
    tools = tools,
    handoffs=handoffs,
    model=gemini_model
)

message = 'Send out a cold sales email addressed to Dear CEO from Alice'

with trace('Automated SDR'):
    result = await Runner.run(sales_manager, message)

#### Check out the trace

In [38]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool 
    name : str

guardrail_agent = Agent(
    name = 'Name check',
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model = gemini_model
)

In [39]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context) 
    is_name_in_message = result.final_output.is_name_in_message 
    return GuardrailFunctionOutput(output_info ={"Found_name": result.final_output}, tripwire_triggered=is_name_in_message)


In [42]:
careful_sales_manager = Agent(
    name = 'Sales Manager',
    instructions=sales_manager_instructions,
    tools = tools,
    handoffs=[emailer_agent],
    model = gemini_model,
    input_guardrails=[guardrail_against_name]
)

message = "Send out a cold sales email addressed to Dear CEO from Dileep"

with trace('Protected Automated SDR'):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [43]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Department"
with trace('Protaced Automated STD'):
    result = await Runner.run(careful_sales_manager, message)